## Arquitectura IoT Protocolo MQTT

![01.png](img/01.png)


### Conceptos

* Funciona sobre TCP
* Usa el puerto **1883**
* Puede usar usuario y password para el mensaje
* Usa SSL/TLS para seguridad

![02.png](img/02.png)

### Elementos clave

* Broker: Es el servidor MQTT
* Publish/Subscribe: Son las acciones del cliente (un cliente puede publicar en el servidor o puede suscribirse en el servidor)
* Topics: Los temas a los cuales se suscribirán y publicarán los clientes. Un cliente puede publicar en un determinado topic o puede suscribirse a un determinado topic, en terminos prácticos, los topics son palabras o temas, por ejemplo temperatura o humedad

![03.png](img/03.png)

**El broker por el momento lo vamos a instalar en un servidor o en la PC.**

![04.png](img/04.png)

El cliente **A** publica en un topic, por ejemplo el topic **prueba**, de tal forma que el cliente **B** se subscribe al topic **prueba** y de esta manera, todo lo que el cliente **A** publique al topic **prueba** (cada vez) le llegará al cliente **B** y a todos los sobscriptores del topic **prueba**. La condición es que tienen que estar en el mismo topic.

![05.png](img/05.png)



## Ejemplo 01 Instalar el servidor MQTT en la PC y hacer pruebas


Para instalar mosquito buscar en google **mosquitto**

![06.png](img/06.png)

***
```
Aplica para windows

Observación 1

Ir al archivo mosquitto.conf

Editar el archivo y adicionar al final 
listener 1883 0.0.0.0
allow_anonimous true

Reiniciar

Observación 2

Acciones en el CMD Windows

cd ..
cd ..
cd "Program Files"
cd mosquitto
mosquitto -c mosquitto.conf
mosquitto_sub -h ipBroker -t topic
mosquitto_pub -h ipBroker -m "mensaje" -t topic


```
***

### Instalación de Mosquitto en Debian 12


`sudo apt update && sudo apt upgrade -y`

`sudo apt install mosquitto mosquitto-clients -y`

`sudo nano /etc/mosquitto/mosquitto.conf`

**mosquitto.conf**
***
```
# Place your local configuration in /etc/mosquitto/conf.d/
#
# A full description of the configuration file is at
# /usr/share/doc/mosquitto/examples/mosquitto.conf.example

pid_file /run/mosquitto/mosquitto.pid

persistence true
persistence_location /var/lib/mosquitto/

log_dest file /var/log/mosquitto/mosquitto.log

include_dir /etc/mosquitto/conf.d



###  AGREGAR ESTAS LINEAS AL FINAL DEL ARCHIVO ###
listener 1883 0.0.0.0
allow_anonymous true
#password_file /etc/mosquitto/passwd

```
***

`sudo systemctl restart mosquitto`

`sudo systemctl status mosquitto`

`sudo systemctl enable mosquitto`


La configuración anterior usa mosquito sin que se establezca un password, para información sonbre la instalación con password, vea la siguiente liga al documento pdf.

<a href="docs/GuíaparaInstalarMosquittoMQTTenDebian12.pdf">Guia de Instalación Mosquitto en Debian 12</a>


### Probar mosquitto

En una terminal usando la dirección ip que tenga su computadora, ejecutar

`mosquitto_sub -h 192.168.2.186 -t "test/topic"`

La terminal se queda bloqueada para recibir los mensajes.

***
```
21:25 $ mosquitto_sub -h 192.168.2.168 -t "test/topic"
```
***


En otra terminal ejecutar el siguiente comando para publicar en el topico, recuerde usar la direccion ip que tiene su computadora.

`mosquitto_pub -h 192.168.2.186 -t "test/topic" -m "Hola Mundo"`

***
```
21:31 $ mosquitto_pub -h 192.168.2.186 -t "test/topic" -m "Hola Mundo"
21:32 $
```
***

Ahora si regresamos a la terminal en donde fue realizada la subscripción se podrá visualizar el mensaje.

***
```
21:25 $ mosquitto_sub -h 192.168.2.186 -t "test/topic"
Hola Mundo
```
***

Se puede detener el comando usando la combinación de teclas **"ctrl+c"**

Es importante realizar correctamente el uso de las direcciones ip.


## Implementación de MQTT en la nube


### Amazon AWS

Amazon AWS va a dar por un año una serie de servicios gratis (free tier) con un consumo limitado de su uso. El servicio que interesa es E2C, de las cuales crearemos instancias (servidores virtuales), Key pair loggin (generar llaves para conexion, .pem para openSSH o .ppk para usar con PuTTY y cambiar de formatos entre llaves usando **puttygen.exe**), el cual no se comparte con nadie




### Oracle Cloud Infraestructure

Asumiendo que ya tenemos una maquina virtual en la nube y que accederemos usando la consola de Linux usando SHH

![07.png](img/07.png)

La dirección del servidor que se usa para pruebas en este momento es 159.54.134.99

Procedemos a realizar lo siguiente:


***
```
$sudo su
#apt-get update
#apt-get upgrade

Nota: en el caso de que se haya actualizado el kernel
#reboot

$sudo su
#apt-get install mosquitto

```
***

Ahora editamos el archivo de configuración de mosquitto

***
```
$sudo su
#cd /etc/mosquitto
#ls
#nano mosquitto.conf
```
***

Agregar las dos lineas al final del archivo, asi que el contenido del archivo se debe ver como:

**mosquitto.conf**

***
```
# Place your local configuration in /etc/mosquitto/conf.d/
#
# A full description of the configuration file is at
# /usr/share/doc/mosquitto/examples/mosquitto.conf.example

pid_file /run/mosquitto/mosquitto.pid

persistence true
persistence_location /var/lib/mosquitto/

log_dest file /var/log/mosquitto/mosquitto.log

include_dir /etc/mosquitto/conf.d

listener 1883 0.0.0.0
allow_anonymous true
```
***



Ahora verificamos el contenido del archivo y reiniciamos el servicio de mosquitto

***
```
#cat mosquitto.conf
#/etc/init.d/mosquitto stop
#/etc/mosquitto# /etc/init.d/mosquitto start
```
***

En este momento, el servicio de mosquitto ya esta funcionando, pero al hacer la prueba esto no va a funcionar aún.

Desde la consola de una maquina diferente que tenga instalado mosquitto, vamos a ejecutar lo siguiente:

***
```
23:29 $ mosquitto_sub -h 159.54.134.99 -t "test/topic"
Error: Connection timed out
```
***

Como se puede observar, ahora da un error, puesto que no se puede conectar ya que la seguridad de la nube de oracle y el firewall lo está bloqueando. Para resolver lo anterior hay que **Abrir puertos**


Loguearse en la cuenta de cloud.oracle.com.

Seleccionar "Dashboard."

![08.png](img/08.png)

Seleccionar el virtual cloud network.

![09.png](img/09.png)

Seleccionar la subnet correspondiente.

![10.png](img/10.png)

Seleccionar el security list.

![11.png](img/11.png)

Seleccionar "Add Ingress Rules"

![12.png](img/12.png)

Realizar lo siguiente:

1. En la nueva regla, no active ni de click "Stateless." 
2. Establezca "Source type" en CIDR. Establezca "Source CIDR" to 0.0.0.0/0. Es posible establecer algo menos general que 0.0.0.0/0, si deseamos restringir las direcciones ip de entrada a nuestro servidor, pero esa sería una configuración para un escenario particular.
3. Establezca el protocolo IP deseado, en este caso, TCP. 
4. Deje establecido Source Port Range en All. 
5. Establezca Dest Port Range al rango de puertos que desea abrir, en este caso, 1883. 
6. Ingrese la descripción de la regla, 


![13.png](img/13.png)

7. Guarde la nueva regla de ingreso. Damos clck en el boton de "Add Ingress Rules"

Ahora esa regla debe aparecer en el dashboard

![14.png](img/14.png)

Una vez creada la regla, Tambien se debe configurar la máquina virtual, ya que incorporan firewalls internos (ya sea iptables o uncomplicated firewall, ufw) y se deben abrir los puertos, pero una vez realizado esto, el servicio funcionará.

Ahora desde dentro de la maquina virtual (por ser la infraestructura de oracle, usa iptables), abrimos el puerto con IPTABLES

**sudo iptables -I INPUT 6 -m state --state NEW -p tcp --dport 1883 -j ACCEPT**

Con esta regla ya deberíamos poder comunicarnos con el servidor de mosquitto.

Para guardar la regla de forma permanente ejecutar los siguientes comandos:

**sudo apt install iptables-persistent -y**

**sudo netfilter-persistent save**


( LO DE ESTOS PARÉNTESIS NO REALIZARLO EN ORACLE CLOUD

NOTA: EN CASO DE USAR otro servicio de nube, que utilice ufw:

firewall-cmd --permanent --zone=public --add-port=1883/tcp; firewall-cmd --reload

)


Hacemos la prueba desde otra consola en otra maquina:

00:26 $ mosquitto_sub -h 159.54.134.99 -t "test/topic"

No habrá ningún mensaje porque al conectarse se queda en modo bloqueante esperando recibir mensajes.


![15.png](img/15.png)


Ahora desde otra terminal en la maquina desde la que se hacen pruebas ejecutamos 

00:26 $ mosquitto_pub -h 159.54.134.99 -t "test/topic" -m "ya esta funcionando"
00:27 $ 

![16.png](img/16.png)

Y podemos ver en la primer terminal, en la que se quedo en modo bloqueante lo siquiente:

00:26 $ mosquitto_sub -h 159.54.134.99 -t "test/topic"

ya esta funcionando

![17.png](img/17.png)

Dicha terminal seguirá en modo bloqueante esperando recibir mas mensaje y con eso aseguramos que el servicio está activo. Podemos terminar la subscripcion usando la combinación de teclas "Ctrl+c".

Ahora el Broker está en la nube. Recuerde que se tendrán que hacer configuraciones adicionales para que el servicio de mosquitto se ejecute cada vez que se reincie la máquina virtual y ademas tambien la regla de iptables requiere mas configuración en caso de que deseemos que sea permanente.


## Instalar Libreria umqtt


Para instalar la librería umqtt en la pico, en caso de que no esté instalada

1. Instalar micropython-upip en Thonny, lo que permite manejar los paquetes.

![18.png](img/18.png)

2. Hacer que el Raspberry Pico W tenga internet (libreria network), para lo cual ejecutamos el código del archivo **06_ejemplo_02_conexion_wifi.py**

![19.png](img/19.png) ofuscar informacion de red en la imagen


Previo al siguiente paso, se puede observar que dentro del directorio lib, no se encuentra la libreria de umqtt

![20.png](img/20.png) ofuscar informacion de red en la imagen

3. En la shell de Thonny
   >>>import mip
   >>>mip.install("umqtt.robust")
   >>>mip.install("umqtt.simple")
   
![21.png](img/21.png) ofuscar informacion de red en la imagen
   
4. Verificar que en la carpeta lib este umqtt del Raspberry Pi Pico W

![22.png](img/22.png) ofuscar informacion de red en la imagen


### Descripción de la librería umqtt


***
```
//Librería
import umqtt.simple as mqtt

//Objeto
c=mqtt.MQTTClient("clienteid", brokermqtt)

//Funciones publicador y suscriptor
c.connect()
c.disconnect()

//Funciones publicador
c.publish(topic, mensaje)

//Funciones de suscriptor
def funcionC(topic, msg):
    print(topic+ " " + msg)

//El proceso en orden es el siguiente

c.callback(funcionC) #definir la funcion

#aqui va el c.connect()
c.subscribe(topic) #subscriber al topic
c.wait_msg() #esperar 1 mensaje, si se desea se puede poner dentro de un bucle para que se quede espereando mensajes.
```
***



## Ejemplo 01 MQTT

Realizar un programa en el Raspberry Pi Pico que se suscriba a un servidor MQTT en el topic "prueba". Luego hacer las pruebas de enviar mensaje desde la App IoT MQTT Panel


**07_ejemplo_01_mqtt_recepcion_mensajes.py**


***
```Python

import network
import utime
import umqtt.simple as mqtt

### Funcion a realizar cuando se reciba un mensaje mqtt
def receptor(topic, msg):
    #print(topic + " " + msg) #imprimir topic y mensaje
    print(msg) # imprimir solo mensaje


### SECCION DE CONEXION CON wifi
wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect('ISSD', 'password red')


# la conexión puede demorar unos segundos,
# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())

### SECCION DE CONEXION CON mqtt
#el id "pico1" debe ser único, si hay mas picos, estos ids deben ser diferentes
#la ip del broker 159.54.134.99
c=mqtt.MQTTClient("pico1", "159.54.134.99")
#definimos la funcion a realizar cuando recibimos un mensaje
c.set_callback(receptor)
#conectamos y suscribimos
c.connect()
c.subscribe('prueba')
#ciclamos la recepcion de mensajes
print("Ejecutando el suscriptor")
while True:
    c.wait_msg()


```
***

<video width="320" height="240" controls>
  <source src="video/01.webm" type="video/mp4">
Your browser does not support the video tag.
</video>

## Probar desde el teléfono 

Buscar la App **iot mqtt panel** en el **PlayStore**  de Google, del autor Rahul Kundu.

Al abrir la aplicación indica que no hay ninguna conexion

![23.jpeg](img/23.jpeg)

En este ejemplo, se agrega la información de la imagen pero recuerde que debe establecer la información correspondiente a su configuración personal.

![24.jpeg](img/24.jpeg)


Se debe dar clic en agregar un dashboard

![25.jpeg](img/25.jpeg)

Y dar clic en el botón de **CREATE**

Cuando el simbolo es color naranja, significa que se ha establecido la conexión. Ahora entramos al dashboard 


![26.jpeg](img/26.jpeg)

Agregar un panel

![27.jpeg](img/27.jpeg)

y agregamos un control (hay controles que son publicadores, como text input, y suscriptores, como text log) para poder enviar información, seleccionamos text input

![28.jpeg](img/28.jpeg)

Se establece el nombre como **texto**, el topic como **prueba**

![29.jpeg](img/29.jpeg)

Y se selecciona el botón de **CREATE**

Escribimos el mensaje **hola desde celular** y lo enviamos

![29.jpeg](img/29.jpeg)

Escribimos el mensaje a enviar

![30.jpeg](img/30.jpeg)

Y se verifica que haya sido recibido en la Raspberry Pi Pico W

![31.png](img/31.png)



## Ejemplo 02 MQTT


Hacer un programa en el Raspberry Pi Poco W que se suscriba a un servidor MQTT en el topic "prueba"

Conectar un led en el GPIO 18, hacer que si recibe la letra A encienda el led, con la letra B apaga el led. En el App crear dos botones y publicar A con el boton "Encender" y publicar B con el botón "Apagar"


![32.png](img/32.png)


**07_ejemplo_02_mqtt_encender_led.py**


***
```Python

import network
import utime
import umqtt.simple as mqtt
from machine import Pin


### Funcion a realizar cuando se reciba un mensaje mqtt
def receptor(topic, msg):
    #print(topic + " " + msg) #imprimir topic y mensaje
    # print(msg) # imprimir solo mensaje
    # print(msg.decode()) # Lo convierte a string
    m = msg.decode() # mensaje en string
    print(m)
    if m == 'A':
        led.on()
    elif m == 'B':
        led.off()
    


### SECCION DE CONEXION CON wifi
wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect('ISSD', 'password red')



# la conexión puede demorar unos segundos,
# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())


### SECCION led
# led de la tarjeta pi pico
led = Pin('LED', Pin.OUT)
# led conectado al pin 18
#led = Pin(18, Pin.OUT)



### SECCION DE CONEXION CON mqtt
#el id "pico1" debe ser único, si hay mas picos, estos ids deben ser diferentes
#la ip del broker 159.54.134.99
c=mqtt.MQTTClient("pico1", "159.54.134.99")
#definimos la funcion a realizar cuando recibimos un mensaje
c.set_callback(receptor)
#conectamos y suscribimos
c.connect()
c.subscribe('prueba')
#ciclamos la recepcion de mensajes
print("Ejecutando el suscriptor")
print("Esperando mensaje")
while True:
    c.wait_msg()


```
***


Agregar un nuevo panel en la aplicación **IoT MQTT Panel** dando click en el botón **+**

![33.jpeg](img/33.jpeg)

Selección **Button**

![34.jpeg](img/34.jpeg)

Configurar el botón, con el atributo Panel name **Encender**, Tópic **prueba**,  Payload **A**, 

![35.jpeg](img/35.jpeg)

todos los demas atributos se dejan como estan, y posteriormente dar clic en **Aceptar**

![36.jpeg](img/36.jpeg)

Ahora se creará el botón para apagar el led. 

Configurar el botón, con el atributo Panel name **Apagar**, Tópic **prueba**,  Payload **B**, 

![37.jpeg](img/37.jpeg)

todos los demas atributos se dejan como estan, y posteriormente dar clic en **Aceptar**

![38.jpeg](img/38.jpeg)


<video width="320" height="240" controls>
  <source src="video/01.webm" type="video/mp4">
Your browser does not support the video tag.
</video>

## Ejemplo 03 MQTT


Hacer un programa en el Raspberry Pi Poco W que pida un mensaje por consola, si el mensaje es "z", terminar el programa, si es otro mensaje publicarlo en el servidor MQTT. En la App adicionar un control "Text Log" que se suscriba al mismo topic del Raspberry Pi Pico W, monitorear los mensajes que publica el Raspberry Pi Pico W.


**07_ejemplo_03_mqtt_publicador_mensajes.py**


***
```Python

import network
import utime
import umqtt.simple as mqtt



### SECCION DE CONEXION CON wifi
wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect('ISSD', 'password red')


# la conexión puede demorar unos segundos,
# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())

### SECCION DE CONEXION CON mqtt
#el id "pico1" debe ser único, si hay mas picos, estos ids deben ser diferentes
#la ip del broker 159.54.134.99
c=mqtt.MQTTClient("pico1", "159.54.134.99")

#conectamos
c.connect()
#ciclamos la recepcion de mensajes
print("Ejecutando el suscriptor")
continuar = True
while continuar:
    a = input('Escriba el mensaje: ')
    if a == 'z':
        continuar = False
    else:
        c.publish('prueba', a)

c.disconnect()
print('Fin del programa')

```
***


Agregar un nuevo panel en la aplicación **IoT MQTT Panel** dando click en el botón **+**

![39.jpeg](img/39.jpeg)

Seleccionar la opción **Text Log**

![40.jpeg](img/40.jpeg)

Configurar el panel con el atributo Panel Name **Log**, el atributo Topic **prueba** y dar click en el botón **Create**

![41.jpeg](img/41.jpeg)



<video width="320" height="240" controls>
  <source src="video/01.webm" type="video/mp4">
Your browser does not support the video tag.
</video>

## Ejemplo 04 MQTT


Hacer un programa en el Raspberry Pi Pico W que realice el monitoreo de la temperatura y humedad de un sensor DHT11, conectado en el GPIO18, crear el App con 2 gauge(medidor radial), uno para temperatura, otro para humedad.

![42.png](img/42.png)

***
```

//Librería
import dht

//Objeto
dht11 = dht.DHT11(Pin(x))

dht11.measure()
dht.temperature()
dht11.humidity()

``` 
***


**07_ejemplo_04_mqtt_publicador_temperatura_humedad.py**


***
```Python

import network
import utime
import umqtt.simple as mqtt
import dht
from machine import Pin


### SECCION DE CONEXION CON wifi
wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect('ISSD', 'password red')


# la conexión puede demorar unos segundos,

### DEFINICIÓN DEL OBJETO DE INTERES
dht11 = dht.DHT11(Pin(18))

# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())

### SECCION DE CONEXION CON mqtt
#el id "pico1" debe ser único, si hay mas picos, estos ids deben ser diferentes
#la ip del broker 159.54.134.99
c=mqtt.MQTTClient("pico1", "159.54.134.99")

#conectamos
c.connect()
#ciclamos la recepcion de mensajes
print("Ejecutando el suscriptor")

while True:
    dht11.measure()
    c.publish('prueba', str(dht11.temperature()))
    c.publish('prueba1',str(dht11.humidity()))
    utime.sleep(2) # Envía cada dos segundos

c.disconnect()
print('Fin del programa')


```
***

**Ejecutamos el programa**

Probamos primero usando dos terminales en la computadora (temperatura y humedad respectivamente), una terminal ejecutando el comando:

`mosquitto_sub -h 159.54.134.99 -t "prueba"`

Y otra terminal ejecutando el comando:

`mosquitto_sub -h 159.54.134.99 -t "prueba"`

![43.png](img/43.png)

### **Eliminar el Text log panel creado en el ejemplo anterior**

Agregar un nuevo panel en la aplicación **IoT MQTT Panel** dando click en el botón **+**

Seleccionar la opción **Gauge**, el cual será para la temperatura

![44.png](img/44.jpeg)


Configurar el panel con el atributo Panel Name **Temp**, el atributo Topic **prueba**, el atributo Payload min **0** y el atributo Payload max **50**, dar click en el botón **Create**

![45.png](img/45.jpeg)


**Ahora para la humedad**
Agregar un nuevo panel en la aplicación **IoT MQTT Panel** dando click en el botón **+**
Seleccionar la opción **Gauge**, el cual será para la humedad

![46.png](img/46.jpeg)


Configurar el panel con el atributo Panel Name **Humedad**, el atributo Topic **prueba1**, el atributo Payload min **0** y el atributo Payload max **100**, dar click en el botón **Create**

![47.png](img/47.jpeg)

Quedando el panel de la siguiente manera

![48.png](img/48.jpeg)

<video width="320" height="240" controls>
  <source src="video/01.webm" type="video/mp4">
Your browser does not support the video tag.
</video>

## Ejemplo 05 MQTT


Hacer un programa en el Raspberry Pi Poco W para controlar un servomotor mediante la aplicación con un control de tipo **slider** (barra de desplazamiento). En la App adicionar un control "Slider" que se suscriba al mismo topic del Raspberry Pi Pico W, monitorear los mensajes que publica el Raspberry Pi Pico W y controlar el servomotor.


### **Ahora crearemos primero la aplicación en el App del celular**

Agregar un nuevo panel en la aplicación **IoT MQTT Panel** dando click en el botón **+**

Seleccionar la opción **Slider**

![49.jpeg](img/49.jpeg)

Configurar el panel con el atributo Panel Name **Control.servo**, el atributo Topic **prueba**, el atributo Payload min **0**, al atributo Payload max **180** y al atributo Slider step **2** y dar click en el botón **Create**

![50.jpeg](img/50.jpeg)

Quedando de la siguiente manera (nota, tal vez desee eliminar los paneles creados en el ejemplo anterior, ya que uno de los topic es el mismo y se modificará a medida que se modifique el slider)

![51.jpeg](img/51.jpeg)


**07_ejemplo_05_mqtt_receptor_mensajes_control_servo.py**


***
```Python

import network
import utime
import umqtt.simple as mqtt
from machine import PWM,Pin

### Funcion a realizar cuando se reciba un mensaje mqtt
def receptor(topic, msg):
    angulo = msg.decode() # decodificar el mensaje
    print(angulo)
    
    #calculamos el valor del duty cycle (es decir, la posición del servo)
    dc = 1638.375 + 36.4*int(angulo)
    pwm.duty_u16(int(dc)) 

### SECCION DE CONEXION CON wifi
wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect('ISSD', 'password red')

# la conexión puede demorar unos segundos,
# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())

### SECCION DE CONEXION CON mqtt
#el id "pico1" debe ser único, si hay mas picos, estos ids deben ser diferentes
#la ip del broker 159.54.134.99
c=mqtt.MQTTClient("pico1", "159.54.134.99")

### DEFINIMOS EL OBJETO DEL SERVOMOTOR
pwm = PWM(Pin(7)) # probar con 18

pwm.freq(50) #frecuencia comun de trabajo del servo
             # es decir el periodo T=20ms
######

#definimos la funcion a realizar cuando recibimos un mensaje
c.set_callback(receptor)
#conectamos y suscribimos
c.connect()
c.subscribe('prueba')
#ciclamos la recepcion de mensajes
print("Ejecutando el suscriptor")
print("Esperando mensajes...")
while True:
    c.wait_msg()



```
***

Imagen funcionamiento

![52.png](img/52.png)


<video width="320" height="240" controls>
  <source src="video/02.webm" type="video/mp4">
Your browser does not support the video tag.
</video>